In [39]:
from sklearn.pipeline import Pipeline, make_pipeline
from dask_ml.preprocessing import OneHotEncoder, MinMaxScaler, DummyEncoder
from dask_ml.model_selection import train_test_split
import pandas as pd
import dask.dataframe as dd
#from dask_ml.linear_model import LinearRegression
from sklearn.linear_model import LinearRegression, LogisticRegression



In [40]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

/anaconda3/lib/python3.7/site-packages/distributed/bokeh/core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


Client Scheduler: inproc://192.168.1.12/7860/8 Dashboard: http://localhost:57005/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


In [41]:
day = dd.read_csv('Bike-Sharing-Dataset/day.csv')
hour = dd.read_csv('Bike-Sharing-Dataset/hour.csv')

In [42]:
hour.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [43]:
X = hour.drop(['casual', 'registered', 'cnt','dteday','instant','temp'], axis = 1)
y = hour['cnt']

In [44]:
columns_onehot_encode = ['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit']
scale = ['atemp', 'hum', 'windspeed']
X[columns_onehot_encode] = X[columns_onehot_encode].astype("category").categorize()

In [45]:
length = len(X)
train_n = round(0.8 * length)
test_n = round(0.2 * length)
X_train = X[0:train_n]
X_test = X[train_n+1:length]
y_train = y.loc[0:train_n]
y_test = y.loc[train_n+1:length]

In [46]:
pipe = make_pipeline(DummyEncoder(), LinearRegression())

In [47]:
pipe.fit(X_train,y_train)
pipe.score(X_test, y_test)

0.6314503152210234